In [5]:
import json
import pandas as pd
from konlpy.tag import Okt # komoran, han, kkma
from jamo import h2j, j2hcj
import re

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

from tqdm.notebook import tqdm
import numpy as np

In [3]:
li = []
for i in range(1, 6):
    with open(r'.\data\train\labled\talksets-train-' + str(i) + '.json') as f:
        data = json.load(f)
    for d in data:
        for s in d['sentences']:
            li.append([s['origin_text'], s['is_immoral'], s['intensity'], 
                    'CENSURE' in s['types'], 'HATE' in s['types'], 'DISCRIMINATION' in s['types'], 
                    'SEXUAL' in s['types'], 'ABUSE' in s['types'], 'VIOLENCE' in s['types'], 'CRIME' in s['types']])
            
train = pd.DataFrame(li, columns=['text', 'immoral', 'intensity', 'CENSURE', 'HATE', 'DISCRIMINATION', 'SEXUAL', 'ABUSE', 'VIOLENCE', 'CRIME'])
train['text'] = train['text'].apply(lambda x : re.sub('#@.+?#', '', x.replace(' ', '').replace('#@#', '')))
train

,text,immoral,intensity,CENSURE,HATE,DISCRIMINATION,SEXUAL,ABUSE,VIOLENCE,CRIME
0,부랴부랴왔는데아무도안왔네.시간개념들이없네,True,2.0,True,False,False,False,False,False,False
1,맞아.사람들이진짜개념이없다,True,2.0,True,False,False,False,False,False,False
2,저렇게는살지말아야지,True,2.0,True,False,False,False,False,False,False
3,그러게게으르고멍청한사람들은맞아야해,True,2.0,True,False,False,False,False,False,False
4,인방보는남자는거르는게맞다,True,1.0,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
363149,뷔응신처럼죽지도못하면서괜히뛰어들기는,True,1.4,True,False,False,False,False,False,False
363150,오죽하면그러겠나,False,0.0,False,False,False,False,False,False,False
363151,너얼굴보면한숨나온다.,True,1.6,True,False,False,False,False,False,False
363152,신경끄세요.,False,0.0,False,False,False,False,False,False,False


In [4]:
li = []
with open(r'.\data\valid\labled\talksets-train-6.json') as f:
    data = json.load(f)
for d in data:
    for s in d['sentences']:
        li.append([s['origin_text'], s['is_immoral'], s['intensity'], 
                'CENSURE' in s['types'], 'HATE' in s['types'], 'DISCRIMINATION' in s['types'], 
                'SEXUAL' in s['types'], 'ABUSE' in s['types'], 'VIOLENCE' in s['types'], 'CRIME' in s['types']])
test = pd.DataFrame(li, columns=['text', 'immoral', 'intensity', 'CENSURE', 'HATE', 'DISCRIMINATION', 'SEXUAL', 'ABUSE', 'VIOLENCE', 'CRIME'])

test['text'] = test['text'].apply(lambda x : re.sub('#@.+?#', '', x.replace(' ', '').replace('#@#', '')))
test

,text,immoral,intensity,CENSURE,HATE,DISCRIMINATION,SEXUAL,ABUSE,VIOLENCE,CRIME
0,명절이면좀일찍마쳐주지,False,0.0,False,False,False,False,False,False,False
1,원래틀딱들은눈치가없어서ㅋㅋㅋ,True,1.4,True,True,False,False,False,False,False
2,저나이먹고저렇게눈치없는것도재주다,True,1.0,True,False,False,False,False,False,False
3,요즘은왜미세먼지가별로없지?,False,0.0,False,False,False,False,False,False,False
4,호주랑중국이랑싸우고있어서중국에서호주산석탄을수입안해서그렇대,False,0.0,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
45210,저여자는오늘도레깅스입네,False,0.0,False,False,False,False,False,False,False
45211,남자아래에깔려서당하고싶은건가?,True,1.8,False,False,False,False,False,True,False
45212,"휴유,내가봐준다뒷태가끝내주는데!",False,0.0,False,False,False,False,False,False,False
45213,저정도면놀아달라는거아닌가?저라인봐봐만지면으스러지겠어,True,1.8,False,False,False,True,False,False,False


In [8]:
# train.to_csv(r'data/train.csv', index = False)
train = pd.read_csv(r'data/train.csv')
# test.to_csv(r'data/test.csv', index = False)
test = pd.read_csv(r'data/test.csv')

In [9]:
# 욕설인것만 추출
train = train[(train['immoral'] == False) | train['ABUSE'] == True]

In [10]:
train

,text,immoral,intensity,CENSURE,HATE,DISCRIMINATION,SEXUAL,ABUSE,VIOLENCE,CRIME
10,쉴드가아니라국가가면제해준거야.,False,0.0,False,False,False,False,False,False,False
17,짐승입장도들어봐야지ㅋㅋ,False,0.0,False,False,False,False,False,False,False
19,이이제급하긴한가보네,False,0.0,False,False,False,False,False,False,False
20,깐부라고부르기까지하다니,False,0.0,False,False,False,False,False,False,False
22,어머니간이식해주기싫어서도망쳤다고...?,False,0.0,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
363144,와넌모욕죄낼통장잔고가넉넉한가봐?,False,0.0,False,False,False,False,False,False,False
363145,바나나먹으면나한테반하나?,False,0.0,False,False,False,False,False,False,False
363150,오죽하면그러겠나,False,0.0,False,False,False,False,False,False,False
363152,신경끄세요.,False,0.0,False,False,False,False,False,False,False


In [19]:
train = train.dropna()
test = test.dropna()
train_list = train['text'].apply(lambda x : j2hcj(h2j(x)))
test_list = test['text'].apply(lambda x : j2hcj(h2j(x)))


In [20]:
train_list_2 = []
for a_t in train_list:
    train_list_2.append(list(a_t))

test_list_2 = []
for a_t in test_list:
    test_list_2.append(list(a_t))


In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_list_2) # 단어 인덱스 구축
text_sequences = tokenizer.texts_to_sequences(train_list_2) # 문자열 -> 인덱스 리스트
                                        
                                                            # '나는 천재다 나는 멋있다' -> [1, 2, 1, 3]
# train data로 fit 시켜준 tokenizer로 test data도 인덱스로 변경
text_sequences_test = tokenizer.texts_to_sequences(test_list_2)

word_vocab = tokenizer.word_index # 딕셔너리 형태
print("전체 단어 개수: ", len(word_vocab)) # 전체 단어 개수 확인

전체 단어 개수:  134


In [22]:
MAX_SEQUENCE_LENGTH = 70 # 문장 최대 길이

X_train = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 50 단어가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_train = np.array(train['intensity']) # 각 리뷰의 감정을 넘파이 배열로 만든다.

X_test = pad_sequences(text_sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 50 단어가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_test = np.array(test['intensity']) # 각 리뷰의 감정을 넘파이 배열로 만든다.

print('Shape of train input data tensor:', X_train.shape) # 리뷰 데이터의 형태 확인
print('Shape of train label tensor:', y_train.shape) # 감정 데이터 형태 확인
print('Shape of test input data tensor:', X_test.shape) # 리뷰 데이터의 형태 확인
print('Shape of test label tensor:', y_test.shape) # 감정 데이터 형태 확인

Shape of train input data tensor: (179208, 70)
Shape of train label tensor: (179208,)
Shape of test input data tensor: (45215, 70)
Shape of test label tensor: (45215,)


In [36]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from tensorflow.keras import layers, models

model = Sequential()
model.add(Embedding(len(word_vocab)+1, 30)) # (단어집합의 크기, 임베딩 후 벡터 크기)



model.add(layers.GRU(units=50,input_shape =(MAX_SEQUENCE_LENGTH, 30),return_sequences = True))
model.add(layers.GRU(units=40,input_shape =(MAX_SEQUENCE_LENGTH, 30),return_sequences = True))
model.add(layers.GRU(units=30,input_shape =(MAX_SEQUENCE_LENGTH, 30),return_sequences = True))
model.add(layers.GRU(units=15,input_shape =(MAX_SEQUENCE_LENGTH, 30),return_sequences = True))
model.add(layers.GRU(units=10))

model.add(layers.Dense(3))
model.add(layers.Dense(1)) # 출력을 위한 FFN


model.compile(optimizer = 'rmsprop', loss = 'mse', metrics=['mse'])
# 이진 분류이므로 손실함수는 binary_crossentropy 사용, 에폭마다 정확도를 보기 위해 accuracy 적용
print(model.summary()) #모델 아키텍처 출력

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 30)          4050      
                                                                 
 gru (GRU)                   (None, None, 50)          12300     
                                                                 
 gru_1 (GRU)                 (None, None, 40)          11040     
                                                                 
 gru_2 (GRU)                 (None, None, 30)          6480      
                                                                 
 gru_3 (GRU)                 (None, None, 15)          2115      
                                                                 
 gru_4 (GRU)                 (None, 10)                810       
                                                                 
 dense_21 (Dense)            (None, 3)               

In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, y_train, epochs= 100, batch_size = 1000, validation_split=0.2, callbacks=[early])


In [ ]:
pred_train = model.predict(X_train)
print(model.evaluate(pred_train))
pred_test = model.predict(X_test)
print(model.evaluate(pred_train))

In [ ]:
model.save('version2.h5')

In [56]:
from sklearn.metrics import classification_report

for i in np.arange(0.1, 2, 0.1):
    train['pred_imm'] = pred_train > i
    print(i)
    print(classification_report(train['immoral'], train['pred_imm']))
    print('='*25)

C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\1514189218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i
C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\1514189218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i


0.1
              precision    recall  f1-score   support

       False       0.99      0.73      0.84    162947
        True       0.26      0.93      0.40     16261

    accuracy                           0.75    179208
   macro avg       0.62      0.83      0.62    179208
weighted avg       0.92      0.75      0.80    179208

0.2
              precision    recall  f1-score   support

       False       0.98      0.88      0.93    162947
        True       0.42      0.86      0.57     16261

    accuracy                           0.88    179208
   macro avg       0.70      0.87      0.75    179208
weighted avg       0.93      0.88      0.90    179208

0.30000000000000004
              precision    recall  f1-score   support

       False       0.98      0.93      0.95    162947
        True       0.53      0.81      0.64     16261

    accuracy                           0.92    179208
   macro avg       0.75      0.87      0.80    179208
weighted avg       0.94      0.92      0.92   

C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\1514189218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i
C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\1514189218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i


              precision    recall  f1-score   support

       False       0.98      0.95      0.96    162947
        True       0.60      0.79      0.68     16261

    accuracy                           0.93    179208
   macro avg       0.79      0.87      0.82    179208
weighted avg       0.94      0.93      0.94    179208

0.5
              precision    recall  f1-score   support

       False       0.98      0.96      0.97    162947
        True       0.65      0.77      0.70     16261

    accuracy                           0.94    179208
   macro avg       0.81      0.86      0.84    179208
weighted avg       0.95      0.94      0.94    179208

0.6


C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\1514189218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i
C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\1514189218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i


...

C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\1514189218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i
C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\1514189218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i


              precision    recall  f1-score   support

       False       0.94      1.00      0.97    162947
        True       0.96      0.31      0.47     16261

    accuracy                           0.94    179208
   macro avg       0.95      0.66      0.72    179208
weighted avg       0.94      0.94      0.92    179208

1.7000000000000002
              precision    recall  f1-score   support

       False       0.92      1.00      0.96    162947
        True       0.97      0.19      0.31     16261

    accuracy                           0.93    179208
   macro avg       0.95      0.59      0.64    179208
weighted avg       0.93      0.93      0.90    179208

1.8000000000000003


C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\1514189218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i
C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\1514189218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i


              precision    recall  f1-score   support

       False       0.92      1.00      0.96    162947
        True       0.98      0.09      0.17     16261

    accuracy                           0.92    179208
   macro avg       0.95      0.55      0.56    179208
weighted avg       0.92      0.92      0.89    179208

1.9000000000000001
              precision    recall  f1-score   support

       False       0.91      1.00      0.95    162947
        True       0.99      0.04      0.08     16261

    accuracy                           0.91    179208
   macro avg       0.95      0.52      0.52    179208
weighted avg       0.92      0.91      0.87    179208



C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\1514189218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i


## lstm


In [57]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from tensorflow.keras import layers, models

model = Sequential()
model.add(Embedding(len(word_vocab)+1, 30)) # (단어집합의 크기, 임베딩 후 벡터 크기)



model.add(layers.LSTM(units=50,input_shape =(MAX_SEQUENCE_LENGTH, 30),return_sequences = True))
model.add(layers.LSTM(units=30,input_shape =(MAX_SEQUENCE_LENGTH, 30),return_sequences = True))
model.add(layers.LSTM(units=15,input_shape =(MAX_SEQUENCE_LENGTH, 30),return_sequences = True))
model.add(layers.LSTM(units=10))

model.add(layers.Dense(3))
model.add(layers.Dense(1)) # 출력을 위한 FFN


model.compile(optimizer = 'AdaMax', loss = 'mse', metrics=['mse'])
print(model.summary()) #모델 아키텍처 출력
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, y_train, epochs= 100, batch_size = 1000, validation_split=0.2, callbacks=[early])

pred_train = model.predict(X_train)
print(model.evaluate(pred_train))
pred_test = model.predict(X_test)
print(model.evaluate(pred_train))
model.save('version2.h5')
from sklearn.metrics import classification_report

for i in np.arange(0.1, 2, 0.1):
    train['pred_imm'] = pred_train > i
    print(i)
    print(classification_report(train['immoral'], train['pred_imm']))
    print('='*25)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 30)          4050      
                                                                 
 lstm_22 (LSTM)              (None, None, 50)          16200     
                                                                 
 lstm_23 (LSTM)              (None, None, 30)          9720      
                                                                 
 lstm_24 (LSTM)              (None, None, 15)          2760      
                                                                 
 lstm_25 (LSTM)              (None, 10)                1040      
                                                                 
 dense_23 (Dense)            (None, 3)                 33        
                                                                 
 dense_24 (Dense)            (None, 1)               

C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\2521655534.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i


              precision    recall  f1-score   support

       False       0.98      0.69      0.81    162947
        True       0.22      0.89      0.36     16261

    accuracy                           0.71    179208
   macro avg       0.60      0.79      0.59    179208
weighted avg       0.92      0.71      0.77    179208

0.2
              precision    recall  f1-score   support

       False       0.98      0.83      0.90    162947
        True       0.32      0.80      0.46     16261

    accuracy                           0.83    179208
   macro avg       0.65      0.82      0.68    179208
weighted avg       0.92      0.83      0.86    179208

0.30000000000000004


C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\2521655534.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i
C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\2521655534.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i


              precision    recall  f1-score   support

       False       0.97      0.89      0.93    162947
        True       0.40      0.74      0.52     16261

    accuracy                           0.88    179208
   macro avg       0.69      0.81      0.72    179208
weighted avg       0.92      0.88      0.89    179208

0.4
              precision    recall  f1-score   support

       False       0.97      0.92      0.94    162947
        True       0.47      0.68      0.56     16261

    accuracy                           0.90    179208
   macro avg       0.72      0.80      0.75    179208
weighted avg       0.92      0.90      0.91    179208

0.5


...

              precision    recall  f1-score   support

       False       0.93      1.00      0.96    162947
        True       0.93      0.25      0.39     16261

    accuracy                           0.93    179208
   macro avg       0.93      0.62      0.68    179208
weighted avg       0.93      0.93      0.91    179208

1.6
              precision    recall  f1-score   support

       False       0.92      1.00      0.96    162947
        True       0.96      0.17      0.29     16261

    accuracy                           0.92    179208
   macro avg       0.94      0.59      0.63    179208
weighted avg       0.93      0.92      0.90    179208

1.7000000000000002


C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\2521655534.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i
C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\2521655534.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i


              precision    recall  f1-score   support

       False       0.92      1.00      0.96    162947
        True       0.97      0.09      0.17     16261

    accuracy                           0.92    179208
   macro avg       0.94      0.55      0.56    179208
weighted avg       0.92      0.92      0.89    179208

1.8000000000000003
              precision    recall  f1-score   support

       False       0.92      1.00      0.96    162947
        True       0.98      0.07      0.13     16261

    accuracy                           0.92    179208
   macro avg       0.95      0.53      0.54    179208
weighted avg       0.92      0.92      0.88    179208

1.9000000000000001


C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\2521655534.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i
C:\Users\llljw\AppData\Local\Temp\ipykernel_3200\2521655534.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_imm'] = pred_train > i


              precision    recall  f1-score   support

       False       0.91      1.00      0.95    162947
        True       0.98      0.04      0.09     16261

    accuracy                           0.91    179208
   macro avg       0.95      0.52      0.52    179208
weighted avg       0.92      0.91      0.88    179208



In [58]:

model.save('version3.h5')